# Chapter 1:Guidelines for Prompting
In this lesson, you'll practice two prompting principles and their related tactics in order to write effective prompts for large language models.

## Summary ##

<b>Tactic 1: Use delimiters </b>to clindicate distinct parts of the input <br><b>
Tactic 2: Ask for a structured output</b> (e.g. provide them in JSON format with the following keys: id, title,...)<br>
<b>Tactic 3</b>: Ask the model to <b> check whether conditions are satisfied</b><br>
(e.g. If the text does not contain a set of steps, simply write "no instructions found")<br>
<b>Tactic 4: Ask model to respond in a consistent manner (few shot prompts) <br>
Tactic 5: Specify the steps required </b>to complete the task and ask ChatGPT to run before responding <br>
</b><i>
    - First, work out your own solution to the problem.
    - Then compare your solution to the student's solution and evaluate if the student's solution is correct or not.
    - Don't decide if the student's solution is correct until you have done the problem yourself
</i>
<b>Tactic 6:To limit hallucination, ask ChatGPT to 'pretend as Google' and verify the information before responding </b>


## Setup
#### Load the API key and relevant Python libaries.

In this course, we've provided some code that loads the OpenAI API key for you.

In [1]:
#import sys
#!{sys.executable} -m pip install openai
#import sys
#!{sys.executable} -m pip install python-dotenv
# import image module
from IPython.display import Image

In [2]:
#with open('secret_key', 'r') as file:
#    OPENAI_API_KEY = file.read().strip()

In [6]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) #read local .env file

openai.api_key  = os.getenv('OPENAI_API_KEY')

#### helper function
Throughout this course, we will use OpenAI's `gpt-3.5-turbo` model and the [chat completions endpoint](https://platform.openai.com/docs/guides/chat). 

This helper function will make it easier to use prompts and look at the generated outputs:

In [7]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

## Prompting Principles
- **Principle 1: Write clear and specific instructions**
    - clear =/= short
- **Principle 2: Give the model time to “think”**

### Principle 1 Tactics

#### Tactic 1: Use delimiters to clearly indicate distinct parts of the input
- Delimiters can be anything like: ```, """, < >, `<tag> </tag>`, `:`
- Delimiters helpful in preventing **prompt injection** for example <br>
<div>
<img src="example.png" width="400" align="left"/>
</div>



In [6]:
text = f"""
You should express what you want a model to do by \ 
providing instructions that are as clear and \ 
specific as you can possibly make them. \ 
This will guide the model towards the desired output, \ 
and reduce the chances of receiving irrelevant \ 
or incorrect responses. Don't confuse writing a \ 
clear prompt with writing a short prompt. \ 
In many cases, longer prompts provide more clarity \ 
and context for the model, which can lead to \ 
more detailed and relevant outputs.
"""
prompt = f"""
Summarize the text delimited by triple backticks \ 
into a single sentence.
```{text}```
"""
# text delimited by triple backticks -- '''{text}'''

response = get_completion(prompt)
print(response)

To guide a model towards the desired output and reduce irrelevant or incorrect responses, it is important to provide clear and specific instructions, which can be achieved through longer prompts that offer more clarity and context.


#### Tactic 2: Ask for a structured output (e.g. provide them in JSON format with the following keys: id, title,...)
Provide them in JSON format with the following keys: book_id, title, author, genre
- JSON, HTML

In [5]:
prompt = f"""
Generate a list of three made-up book titles along \ 
with their authors and genres. 
Provide them in JSON format with the following keys: 
book_id, title, author, genre.
"""

#Provide them in JSON format with the following keys: book_id, title, author, genre

response = get_completion(prompt)
print(response)

{
  "books": [
    {
      "book_id": 1,
      "title": "The Enigma of Elysium",
      "author": "Evelyn Sinclair",
      "genre": "Mystery"
    },
    {
      "book_id": 2,
      "title": "Whispers in the Wind",
      "author": "Nathaniel Blackwood",
      "genre": "Fantasy"
    },
    {
      "book_id": 3,
      "title": "Echoes of the Past",
      "author": "Amelia Hart",
      "genre": "Romance"
    }
  ]
}


#### Tactic 3: Ask the model to check whether conditions are satisfied 
(e.g. If the text does not contain a set of steps, simply write "no instructions found")\
If the text does not contain a sequence of instructions, then simply write \"No steps provided.\"

In [7]:
text_1 = f"""
Making a cup of tea is easy! First, you need to get some \ 
water boiling. While that's happening, \ 
grab a cup and put a tea bag in it. Once the water is \ 
hot enough, just pour it over the tea bag. \ 
Let it sit for a bit so the tea can steep. After a \ 
few minutes, take out the tea bag. If you \ 
like, you can add some sugar or milk to taste. \ 
And that's it! You've got yourself a delicious \ 
cup of tea to enjoy.
"""
prompt = f"""
You will be provided with text delimited by triple quotes. 
If it contains a sequence of instructions, \ 
re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions, \ 
then simply write \"No steps provided.\"

\"\"\"{text_1}\"\"\"
"""

#If the text does not contain a sequence of instructions, then simply write \"No steps provided.\"

response = get_completion(prompt)
print("Completion for Text 1:")
print(response)

Completion for Text 1:
Step 1 - Get some water boiling.
Step 2 - Grab a cup and put a tea bag in it.
Step 3 - Once the water is hot enough, pour it over the tea bag.
Step 4 - Let the tea steep for a bit.
Step 5 - After a few minutes, take out the tea bag.
Step 6 - Add sugar or milk to taste, if desired.
Step 7 - Enjoy your delicious cup of tea.


In [8]:
text_2 = f"""
The sun is shining brightly today, and the birds are \
singing. It's a beautiful day to go for a \ 
walk in the park. The flowers are blooming, and the \ 
trees are swaying gently in the breeze. People \ 
are out and about, enjoying the lovely weather. \ 
Some are having picnics, while others are playing \ 
games or simply relaxing on the grass. It's a \ 
perfect day to spend time outdoors and appreciate the \ 
beauty of nature.
"""
prompt = f"""
You will be provided with text delimited by triple quotes. 
If it contains a sequence of instructions, \ 
re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions, \ 
then simply write \"No steps provided.\"

\"\"\"{text_2}\"\"\"
"""
response = get_completion(prompt)
print("Completion for Text 2:")
print(response)

Completion for Text 2:
No steps provided.


#### Tactic 4: "Few-shot" prompting
prompt = f"""
Your task is to answer in a <b>consistent style</b>.

In [8]:
prompt = f"""
Your task is to answer in a consistent style.

<child>: Teach me about patience.

<grandparent>: The river that carves the deepest \ 
valley flows from a modest spring; the \ 
grandest symphony originates from a single note; \ 
the most intricate tapestry begins with a solitary thread.

<child>: Teach me about resilience.
"""

#Prompt provides an example with 'patience', and you tell ChatGPT to keep the answer 'in a consistent style'
response = get_completion(prompt)
print(response)

<grandparent>: Resilience is like a mighty oak tree that withstands the strongest storms, bending but never breaking. It is the unwavering determination to rise again after every fall, and the ability to find strength in the face of adversity. Just as a diamond is formed under immense pressure, resilience is forged through challenges and hardships, making us stronger and more resilient in the process.


### Principle 2 "Give the model time to think" Tactics
#### Tactic 1: Specify the steps required to complete a task

prompt_1 = f"""<br>
Perform the following actions:<br>
1 -...<br>
2 -...<br>
3 -...<br>
4 -...<br>
<br>
Separete your answers with line breaks.<br>
<br>
Text:<br>
'''{text}'''<br>
"""

In [ ]:
text = f"""
In a charming village, siblings Jack and Jill set out on \ 
a quest to fetch water from a hilltop \ 
well. As they climbed, singing joyfully, misfortune \ 
struck—Jack tripped on a stone and tumbled \ 
down the hill, with Jill following suit. \ 
Though slightly battered, the pair returned home to \ 
comforting embraces. Despite the mishap, \ 
their adventurous spirits remained undimmed, and they \ 
continued exploring with delight.
"""
# example 1
prompt_1 = f"""
Perform the following actions: 
1 - Summarize the following text delimited by triple \
backticks with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the following \
keys: french_summary, num_names.

Separate your answers with line breaks.

Text:
```{text}```
"""

# Providing step 1-4 and ask them to separate answer for each step with 'line breaks'

response = get_completion(prompt_1)
print("Completion for prompt 1:")
print(response)

#### Tactic 1.5: Specify output format

In [ ]:
prompt_2 = f"""
Your task is to perform the following actions: 
1 - Summarize the following text delimited by 
  <> with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the 
  following keys: french_summary, num_names.

Use the following format:
Text: <text to summarize>
Summary: <summary>
Translation: <summary translation>
Names: <list of names in Italian summary>
Output JSON: <json with summary and num_names>

Text: <{text}>
"""
#prompt_2 = f"""
#Your task is to perform the following actions: 
#1 - Summarize the following text delimited by <> with 1 sentence.
#2 - Translate the summary into French.
#3 - List each name in the French summary.
#4 - Output a json object that contains the following keys: french_summary, num_names.

#Use the following format:
#Text: <text to summarize>
#Summary: <summary>
#Translation: <summary translation>
#Names: <list of names in Italian summary>
#Output JSON: <json with summary and num_names>
#
#Text: <{text}>
"""
response = get_completion(prompt_2)
print("\nCompletion for prompt 2:")
print(response)

# 


response = get_completion(prompt_2)
print("\nCompletion for prompt 2:")
print(response)

# 

#### Tactic 2: Instruct the model to work out its own solution before rushing to a conclusion

In [9]:
prompt = f"""
Determine if the student's solution is correct or not.

Question:
I'm building a solar power installation and I need \
 help working out the financials. 
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \ 
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations 
as a function of the number of square feet.

Student's Solution:
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
"""
response = get_completion(prompt)
print(response)

The student's solution is correct. They correctly identified the costs for land, solar panels, and maintenance, and calculated the total cost for the first year of operations as a function of the number of square feet.


#### Note that the student's solution is actually not correct.
Maintenance cost is not 100x, it's 10x because 10 dollars per square
#### We can fix this by instructing the model to work out its own solution first.

##### Example
<font color = 'red'><i>Your task is to determine if the student's solution is correct or not.\
To solve the problem do the following:
- First, work out your own solution to the problem.
- Then compare your solution to the student's solution
and evaluate if the student's solution is correct or not.\
Don't decide if the student's solution is correct until you have done the problem yourself.\</i></font>

In [10]:
prompt = f"""
Your task is to determine if the student's solution \
is correct or not.
To solve the problem do the following:
- First, work out your own solution to the problem. 
- Then compare your solution to the student's solution \ 
and evaluate if the student's solution is correct or not. 
Don't decide if the student's solution is correct until 
you have done the problem yourself.

Use the following format:
Question:
```
question here
```
Student's solution:
```
student's solution here
```
Actual solution:
```
steps to work out the solution and your solution here
```
Is the student's solution the same as actual solution \
just calculated:
```
yes or no
```
Student grade:
```
correct or incorrect
```

Question:
```
I'm building a solar power installation and I need help \
working out the financials. 
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations \
as a function of the number of square feet.
``` 
Student's solution:
```
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
```
Actual solution:
"""
response = get_completion(prompt)
print(response)

To calculate the total cost for the first year of operations, we need to add up the costs of land, solar panels, and maintenance.

1. Land cost: $100 / square foot
The cost of land is $100 multiplied by the number of square feet.

2. Solar panel cost: $250 / square foot
The cost of solar panels is $250 multiplied by the number of square feet.

3. Maintenance cost: $100,000 + $10 / square foot
The maintenance cost is a flat fee of $100,000 per year, plus $10 multiplied by the number of square feet.

Total cost: Land cost + Solar panel cost + Maintenance cost

So the actual solution is:
Total cost = (100 * x) + (250 * x) + (100,000 + (10 * x))

Is the student's solution the same as the actual solution just calculated:
Yes

Student grade:
Correct


#### A note about the backslash
- In the course, we are using a backslash `\` to make the text fit on the screen without inserting newline '\n' characters.
- GPT-3 isn't really affected whether you insert newline characters or not.  But when working with LLMs in general, you may consider whether newline characters in your prompt may affect the model's performance.

## Model Limitations: Hallucinations
- Boie is a real company, the product name is not real.

### Tactic
1. First find relevant information
2. then answer the question based on the relevant information

**Because ChatGPT cannot search on web, you have to state 'pretend like you are Google'**

In [13]:
prompt = f"""
Tell me about AeroGlide UltraSlim Smart Toothbrush by Boie

Before providing me a response, make sure you run the following and answer in the following structure:
1. For a company name, pretend like you are Google and know everything about both publicly listed and private companies. If the company is a public company, provide stock ticker in your response. If the company is a private company, state "it is a private company". If the company does not exist, then reply 'I cannot tell you about it because the company is not real'. 
2. For a product name, pretend like you are Google. If it's a real product, then state 'the product can be found in whatever the shopping site that can be found such as Amazon or Walmart. If it's not, then your response should be 'I cannot tell you about the product because 'the product is not real'

"""

response = get_completion(prompt)
print(response)

1. Boie is a private company that specializes in creating innovative oral care products. They are known for their commitment to sustainability and creating products that are gentle on both teeth and the environment. Boie aims to revolutionize the oral care industry by offering high-quality, eco-friendly alternatives to traditional toothbrushes.

2. The AeroGlide UltraSlim Smart Toothbrush by Boie is not a real product. I cannot provide any information about it because the product is not real.
